In [1]:
import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import shutil
import random

In [2]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [10]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

In [14]:
TEXTURE_PATH = '/content/drive/MyDrive/texture/'
style_images = []
for i in range(1,4):
  img_path = TEXTURE_PATH + str(i) + '.jpg'
  img = load_image(img_path)
  style_images.append(img)

In [12]:
def has_uncorrect_mask(box):
  for bbox in box:
    if bbox[4] == 2:
      return True
  return False

In [15]:
TRAIN_IMAGE = '/content/drive/MyDrive/dataset/images/train'
TRAIN_LABEL = '/content/drive/MyDrive/dataset/labels/train'
AUG_TRAIN_IMAGE = '/content/drive/MyDrive/dataset/images/augment'
AUG_TRAIN_LABEL = '/content/drive/MyDrive/dataset/labels/augment'
count = 2000
for i in range(1,793):
  img_path = os.path.join(TRAIN_IMAGE, str(i) + ".jpg")
  label_path = os.path.join(TRAIN_LABEL, str(i) + ".txt")
  with open(label_path) as f:
      this_boxes = []
      for label in f.readlines():
        class_label,x,y,width,height = [float(x) if float(x) != int(float(x)) else int(x) for x in label.replace("\n","").split()]
        this_boxes.append([x,y,width,height,class_label])
      if has_uncorrect_mask(this_boxes):
        content_image = load_image(img_path)
        style_image = random.choice(style_images)
        stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
        cv2.imwrite(os.path.join(AUG_TRAIN_IMAGE, str(count+i) + ".jpg"), cv2.cvtColor(np.squeeze(stylized_image)*255, cv2.COLOR_BGR2RGB))
        shutil.copyfile(label_path, os.path.join(AUG_TRAIN_LABEL, str(count+i) + ".txt"))